# Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET acquisitions, where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h
- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

## Execute the `search` operation

In [ ]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pygeofilter_aeronet.utils import verbose_client
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    verbose_client(client)
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

item: Item = Item.from_dict(item_dict)

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(start_datetime)},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(end_datetime)},
            ],
        },
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

2025-11-13 16:34:43.743 | WARNING  | pygeofilter_aeronet.utils:wrapper:71 - GET https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004
2025-11-13 16:34:43.743 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Host: stac.dataspace.copernicus.eu
2025-11-13 16:34:43.744 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Accept: */*
2025-11-13 16:34:43.744 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Accept-Encoding: gzip, deflate
2025-11-13 16:34:43.744 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Connection: keep-alive
2025-11-13 16:34:43.745 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > User-Agent: python-httpx/0.28.1
2025-11-13 16:34:43.745 | WARNING  | pygeofilter_aeronet.utils:wrapper:77 - >
2025-11-13 16:34:44.550 | SUCCESS  | pygeofilter_aeronet.utils:wrapper:99 - < 200 OK
2025-11-13 16:34:44.550 | SUCCESS  | pygeofilter_aerone

KeyError: (<class 'str'>, typing.Any)

In [ ]:
from pygeofilter_aeronet import aeronet_search

item: Item = aeronet_search(
    cql2_filter=cql2_filter,
    output_dir=out_dir,
    verbose=True
)

json.dump(item.to_dict(), sys.stdout, indent=2)

## Visualize the results as Data Frame

In [ ]:
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame

geoparquet_file: str = item.get_assets()['geoparquet'].href
geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

geoparquet_data

## Visualize results on Map screen

In [ ]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import (
    Fullscreen
)
from IPython.display import (
    display,
    HTML
)

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints: GeoJson = GeoJson(
    geoparquet_data.dissolve(by='AERONET_Site').to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map